# Cross-resonance gate between 2 transmons: linear approximation

In [1]:
import numpy as np
import time
from pysqkit import QubitSystem, Qubit
import pysqkit
from pysqkit.util.linalg import get_mat_elem
import qutip
import itertools
from functools import partial
import multiprocessing
import json
import cmath
from IPython.display import display, Latex

In [2]:
control_freq = 5.0

levels_t = 3

transm_c = pysqkit.qubits.SimpleTransmon(
    label='T2', 
    max_freq=control_freq, 
    anharm=-0.3,  
    dim_hilbert=levels_t
)

jc = 0.002 

eps_drive = 0.030

In [3]:
def zz(system: QubitSystem) -> float:
    xi_zz = system.state('00')[0] + system.state('11')[0] \
        - system.state('01')[0] - system.state('10')[0]
    return xi_zz

def mu_yz_control(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'])
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 - yz1))/2

def mu_yi_control(comp_states, op) -> float:
    yz0 = get_mat_elem(op, comp_states['00'], comp_states['10'] )
    yz1 = get_mat_elem(op, comp_states['01'], comp_states['11'] )
    return (np.imag(yz0 + yz1))/2

In [4]:
def cr_gate_time(
    cr_coeff: float
):
    return 1/(2*np.pi*cr_coeff)*np.pi/4

def cr_effect_sw(
    jc: float,
    eps_drive: float,
    transm_t: Qubit,
    transm_c: Qubit
) -> float:
    """
    Description
    ---------------------------------------------------------------------
    Returns the cross-resonance coefficient in the linear approximation
    using the result of Magesan-Gambetta
    """
    j_eff = jc*transm_c.charge_zpf*transm_t.charge_zpf
    delta_t = transm_t.anharm
    delta_c = transm_c.anharm
    detuning = transm_c.freq - transm_t.freq
    return j_eff*transm_c.charge_zpf*eps_drive/detuning*(delta_c/(delta_c + detuning))/2

# Generating data

In [5]:
def get_cr_effect(
    freq: float,
    eps: float,
    jc: float,
    transm_c: Qubit,
):
    """
    Description
    -----------------------------------------------------------------------
    Returns the exact cross-resonance coefficient in MHz.
    """
    
    freq 
    
    transm_t = pysqkit.qubits.SimpleTransmon(
            label='T_tmp', 
            max_freq=freq, 
            anharm=transm_c.anharm,
            dim_hilbert=transm_c.dim_hilbert,
        )
    
    eps_ratio = eps*transm_c.charge_zpf/np.abs(freq - (transm_c.max_freq + transm_c.anharm))
    
    freq_drive = freq
    
    coupled_sys = transm_t.couple_to(transm_c, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
    
    state_label = ["00", "01", "10", "11"]
    comp_states = {}
    for label in state_label:
        state_tmp = coupled_sys.state(label)[1]
        loc = np.argmax(np.abs(state_tmp))
        phase = cmath.phase(state_tmp[loc])
        state_tmp = np.exp(-1j*phase)*state_tmp
        comp_states[label] = state_tmp
    
    mu_cr = mu_yz_control(comp_states, coupled_sys['T2'].charge_op())*eps*1e3/2
    
    mu_cr_sw = cr_effect_sw(jc, eps, transm_t, transm_c)*1e3
    
    zz_coeff = zz(coupled_sys)*1e3
    
    output = {}
    output["freq_fixed"] = transm_t.freq
    output["freq"] = freq
    output["eps"] = eps
    output["eps_ratio"] = eps_ratio
    output["cr"] = np.abs(mu_cr)
    output["cr_sw"] = np.abs(mu_cr_sw)
    output["zz"] = zz_coeff
    output["gate_time"] = cr_gate_time(np.abs(mu_cr/1e3))
    output["gate_time_sw"] = cr_gate_time(np.abs(mu_cr_sw/1e3))
    
    output["jc"] = jc*1e3
    
    return output

func = partial(get_cr_effect, eps=0.03, jc=jc, transm_c=transm_c)
    
func(4.9)

{'freq_fixed': 4.9,
 'freq': 4.9,
 'eps': 0.03,
 'eps_ratio': 0.157619002661481,
 'cr': 0.5166547240467522,
 'cr_sw': 0.5171638967423682,
 'zz': 0.0716855244338177,
 'gate_time': 241.94107627803038,
 'gate_time_sw': 241.7028736680557,
 'jc': 2.0}

In [6]:
delta_freq = 0.8
n_points = 200
freq_list = list(np.linspace(transm_c.freq - delta_freq, transm_c.freq + delta_freq, n_points))

n_process = 4

start = time.time()

pool = multiprocessing.Pool(processes=n_process)

result = pool.map(func, freq_list)

pool.close()
pool.join()

end = time.time()

print("Computation time = {} s".format(end - start))



Computation time = 0.6578915119171143 s


In [7]:
save = True
if save:
    with open('data_cr_transmon_transmon_linear.txt', 'w') as fp:
        json.dump(result, fp)